In [2]:
import re
import gensim
import logging
import pandas as pd
import urllib.request
from gensim.models import word2vec

In [3]:

urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x1a173a0400>)

In [4]:
m = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.KeyedVectors.load(m)

In [8]:
words = ['благодаря_S', 'ночь_S', 'человек_S', 'семантика_S', 'студент_S', 'биткоин_S']

In [9]:
for word in words:
    # есть ли слово в модели? 
    if word in model:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

Увы, слова "благодаря_S" нет в модели!
ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S 0.6946247816085815
утро_S 0.5730193853378296
ноченька_S 0.5582467317581177
рассвет_S 0.5553582906723022
ночка_S 0.5351512432098389
полдень_S 0.5334426760673523
полночь_S 0.478694349527359
день_S 0.4780748784542084
сумерки_S 0.43902185559272766
фундерфун_S 0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681  0.04785006]
женщина_S 0.5979775190353394
парень_S 0.4991787374019623
мужчина_S 0.4767409563064575
мужик_S 0.47383999824523926
россиянин_S 0.4719043970108032
народ_S 0.4654741883277893
согражданин_S 0.45378509163856506
горожанин_S 0.44368088245391846
девушка_S 0.44314485788345337
иностранец_S 0.43849870562553406


семантика_S
[-0.03066749  0.0053851   0.1110732   0.0152335   0.00440643  0.00384104
  0.00096944 -0.03538784 

In [7]:
print(model.doesnt_match('мир_S жизнь_S бытие_S ничто_S'.split()))

мир_S


In [82]:
from pymystem3 import Mystem
f = 'news_corpora.txt'
with open(f, 'r', encoding="utf-8") as file:
    text = file.read()
n_text = re.sub(r"[.,;:!?/\\#@$&)«»“(\"><^%—]", ' ', text)
new_text = re.sub('-\s', '', n_text)
#new_text = n_text.replace('-', '') 
#print(new_text)
m = Mystem()
lemmas = m.lemmatize(new_text)
for_safe = ''.join(lemmas)
#print(for_safe)
name_file = './for_learning.txt'
with open(name_file, 'w+', encoding="utf-8") as fil:
    fil.write(for_safe)

#for_ler = 'for_learning.txt'
#data = gensim.models.word2vec.LineSentence(for_ler)

In [83]:
for_ler = 'for_learning.txt'
data = gensim.models.word2vec.LineSentence(for_ler)

In [84]:
%time model_news = gensim.models.Word2Vec(data, size=300, window=5, min_count=2, workers=2)

CPU times: user 654 ms, sys: 17.3 ms, total: 671 ms
Wall time: 424 ms


In [85]:
model_news.init_sims(replace=True)
model_path = "news.bin"

print("Saving model...")
model_news.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [86]:

print(len(model_news.wv.vocab))

4222


In [87]:
print(sorted([w for w in model_news.wv.vocab]))

['0', '00', '000', '1', '10', '100', '109-й', '11', '12', '1244', '13', '14', '15', '150', '16', '17', '1700', '1710', '1797', '18', '1864', '19', '1921', '1931', '1937', '1977', '1979', '1980', '1985', '1986', '1986U2R', '1989', '1990', '1991', '1992', '1993', '1996', '1998', '1999', '2', '20', '200', '2000', '2001', '2001-м', '2002', '2003', '2004', '2005', '2005-м', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '21', '22', '23', '24', '25', '250', '258', '26', '27', '28', '29', '3', '30', '30-х', '300', '326', '34', '35', '36', '38', '4', '40', '400', '43', '45', '46', '48', '5', '50', '50-летний', '500', '51', '52', '55', '59', '6', '6-7', '60', '600', '62', '67P', '7', '70', '70-летний', '700', '7000', '73', '75', '76', '8', '80-х', '800', '84', '850', '89', '9', '90', '90-е', '90-х', '96', '99', 'Advisors', 'All4Syria', 'AppStore', 'Apple', 'BBC', 'BMW', 'Baboom', 'Bank', 'C02', 'CAVEX', 'CC-BY-SA', 'CG', 'Capital', 'Commons', 'Creative', 'Deutsche', 'Do

In [89]:
print(model_news.wv.most_similar("особенно", topn=10))

[('и', 0.9999284744262695), ('на', 0.9999279379844666), ('с', 0.9999277591705322), ('этот', 0.9999274015426636), ('все', 0.9999269247055054), ('они', 0.9999263286590576), ('человек', 0.9999263286590576), ('по', 0.9999261498451233), ('из', 0.9999260902404785), ('становиться', 0.999925971031189)]
